In [1]:
from google.colab import drive
drive.mount('/content/drive' , force_remount = True)

Mounted at /content/drive


In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import random
from tqdm import tqdm
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn import pipeline
from sklearn.feature_selection import SelectKBest, f_regression, f_classif, chi2, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from scipy.stats import pearsonr
from sklearn.metrics import confusion_matrix, classification_report, silhouette_score
from sklearn.svm import SVR, LinearSVR, SVC, LinearSVC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from scipy import stats
from keras.layers import Dense
from keras.models import Sequential
from sklearn.metrics import plot_confusion_matrix, auc
from sklearn.impute import SimpleImputer

In [7]:
def readDataAndGetSateData():
  #combined_data = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/5.ADI_Analysis/final_data/combined_data_temporary_unverified_pop.pickle','rb'))

  # in mtp Drive 
  #combined_data = pickle.load(open('/content/drive/MyDrive/final_data/combined_data_temporary_unverified_pop.pickle','rb'))
  #drop rows where atleast one column value in Nan or not there

  # now use only latest dataframe
  combined_data = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/Latest_script_Data_for_hyothesis_testing_March_02_2022/final_combined_without_temporal_correction_arch2_predicted_ADI_for_2001_2019_ground_truth_ADI_2011_variables_predicted_from_arch2_pop_sc_st_lit_village_data.pkl','rb'))

  combined_data = combined_data.dropna()

  combined_data["STATE_CODE_2011"] = combined_data["STATE_CODE_2011"].astype(np.int64)
  combined_data["DIST_CODE_2011"] = combined_data["DIST_CODE_2011"].astype(np.int64)
  combined_data["STATE_CODE_2001"] = combined_data["STATE_CODE_2001"].astype(np.int64)
  combined_data["DIST_CODE_2001"] = combined_data["DIST_CODE_2001"].astype(np.int64)

  combined_data_state_group = combined_data.groupby("STATE_CODE_2011")

  print("Number of States: "+str(len(combined_data_state_group)))

  return combined_data_state_group

In [4]:
def giniIndex(cummulative_adi_product_list , cummulative_pop_list):
  '''
  if A + B is the area below the curve of Perfect Equality Line and B is the area below the Lorenz Cruve
  then Gini Index is given by A / A + B
  '''

  diffArea_A = 0
  totalArea_A_and_B = 0

  
  
  from sklearn import metrics
  
  
  
  totalArea_A_and_B = metrics.auc(np.array(cummulative_pop_list), np.array(cummulative_pop_list))
  diffArea_A = totalArea_A_and_B - metrics.auc(np.array(cummulative_pop_list), np.array(cummulative_adi_product_list))


  gini = diffArea_A / totalArea_A_and_B #* 100

  print("Gini Co-Effecient trapezium Rule : "+str(gini))

  return gini

In [5]:
def plot2(data_frame,year,dir,dist_id,enable_plot = True,generate_df = False,state_id = None):
  num_villages = len(data_frame)

  df_gini = None
  
  '''
  # added for dummy
  global dummy_local_dataframe
  # added for dummy
  '''

  data_frame = data_frame[['ADI_'+year,'total_population_'+year]] 

  data_frame['ADI_product_Population_'+year] = data_frame['total_population_'+year] * data_frame['ADI_'+year]


  total_sum_ADI_product_Population_var_name = "total_sum_ADI_product_Population_"+year;
  vars()[total_sum_ADI_product_Population_var_name] = data_frame['ADI_product_Population_'+year].sum()


  total_sum_Population_var_name = "total_sum_Population_"+year;
  vars()[total_sum_Population_var_name] = data_frame['total_population_'+year].sum()

  data_frame = data_frame.groupby(['ADI_'+year],as_index=False).sum()

  data_frame = data_frame.sort_values(by=['ADI_'+year], ascending=True)

  

  adi_product_pop_sum = 0;
  pop_sum = 0;

  # As it starts with zero , for starting meeting point in the curve
  cummulative_adi_product_list = [0]
  cummulative_pop_list = [0]

  for i in list(data_frame.index):
      current_adi = data_frame.loc[i,'ADI_'+year]
      current_population = data_frame.loc[i,'total_population_'+year]
      adi_product_pop_sum += (current_adi *  current_population)
      pop_sum += current_population

      cummulative_adi_product_list.append( adi_product_pop_sum )
      cummulative_pop_list.append( pop_sum )

  if(vars()[total_sum_Population_var_name] != pop_sum):
      print("Poplulation Not Same Some Issue");
      quit()

  if(vars()[total_sum_ADI_product_Population_var_name] != adi_product_pop_sum):
      print("ADi * population  Not Same Some Issue");
      quit()

  cummulative_adi_product_list = cummulative_adi_product_list/adi_product_pop_sum * 100

  cummulative_pop_list = cummulative_pop_list / pop_sum * 100


  data_frame['Cummulative_adi_product_Percent'] = cummulative_adi_product_list[1:]
  data_frame['Cummulative_Population_Percent'] = cummulative_pop_list[1:]

  '''
  # added for dummy
  
  dummy_local_dataframe = data_frame.copy()
  # added for dummy
  '''

  print(data_frame[['ADI_'+year,'Cummulative_Population_Percent','Cummulative_adi_product_Percent']])


  if(len(data_frame) > 1):  
    gini = giniIndex(cummulative_adi_product_list , cummulative_pop_list) 
  else : 
    print("Gini 0.0 as there is only one ADI in a district count: "+str(len(data_frame)))
    gini = 0.0;

  print("[2011 State Id]: "+str(state_id)+"  [2011 District Id]: "+str(dist_id))

  if enable_plot and len(data_frame) > 1:
      f, axArr = plt.subplots(nrows=1,ncols=3,figsize=(40,15))


      pd.Series(cummulative_adi_product_list).plot(ax=axArr[0],kind="bar",title="Cummulative ADI Product") 
      #plt.title('Cummulative ADI Product')
      pd.Series(cummulative_pop_list).plot(ax=axArr[1],kind="bar",title="Cummulative Population")
      #plt.title('Cummulative Population')


      axArr[2].plot(cummulative_pop_list, cummulative_adi_product_list, color='green',label='Lorenz Cruve')
      axArr[2].plot(cummulative_pop_list , cummulative_pop_list, color='red',label='Prefect Equality Line')
      plt.xlabel("Cummulative Population Percentage in a District" , fontsize=20)
      plt.ylabel("Cummulative ADI * Population Percentage in a District" , fontsize=20)
      plt.legend(fontsize=20)
      

      plt.savefig(dir+'/2011_district_id_'+str(int(dist_id))+'_gini_'+str(round(gini,4))+'_.png')
      plt.show()

  if  generate_df : 
      df_gini = pd.DataFrame(columns=["STATE_CODE_2011","DIST_CODE_2011","Gini_"+year,"Population_sum_"+year,"Adi_product_population_sum_"+year,"Average_ADI_per_population_"+year,"Num_villages_in_District_"+year])
      #df_gini.set_index(['STATE_CODE_2011','DIST_CODE_2011'])
      df_gini = df_gini.append({          
                "STATE_CODE_2011":state_id,
                "DIST_CODE_2011":dist_id,
                "Gini_"+year : gini,
                "Population_sum_"+year : pop_sum,
                "Adi_product_population_sum_"+year: adi_product_pop_sum,
                "Average_ADI_per_population_"+year: (adi_product_pop_sum / pop_sum),
                "Num_villages_in_District_"+year : num_villages 
                  }, ignore_index=True) 

   
  return df_gini


In [6]:

def getYearDataframe(year):
    data_state_group = readDataAndGetSateData()
    df_old = pd.DataFrame(columns=["STATE_CODE_2011","DIST_CODE_2011","Gini_"+year,"Population_sum_"+year,"Adi_product_population_sum_"+year,"Average_ADI_per_population_"+year,"Num_villages_in_District_"+year])
    df_old.set_index(['STATE_CODE_2011','DIST_CODE_2011'])
     #dummy dir
    state_dir = None

    for key,item in data_state_group:
      
      
      # for start from in_between due to issue
      #if key != 9:
      #   continue

      local_state = data_state_group.get_group(key)


      data_district_group = local_state.groupby("DIST_CODE_2011")
      
      for key_d,item_d in data_district_group:
          
          local_district = data_district_group.get_group(key_d)
          
          #ignore district with less than 2 villages
         
          df_new = plot2(local_district,year,state_dir,key_d,enable_plot=False,generate_df=True,state_id=key)   
          frames = [df_old,df_new]
          df_old = pd.concat(frames , ignore_index=True)
          #df_old.set_index(['STATE_CODE_2011','DIST_CODE_2011'])

         

      print("Completed state Id : "+str(int(key)))    
    print("Successfully Completed Everything")

    return df_old

In [8]:
df_2001 = getYearDataframe("2001")

Number of States: 14
   ADI_2001  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         7                        1.078152                         0.590754
1         8                        2.164537                         1.271058
2         9                        8.576073                         5.787885
3        10                       11.429640                         8.021544
4        11                       28.129993                        22.401159
5        12                       31.031360                        25.126449
6        13                       69.693205                        64.468302
7        14                       70.367029                        65.206722
8        15                      100.000000                       100.000000
Gini Co-Effecient trapezium Rule : 0.08198681901838427
[2011 State Id]: 3  [2011 District Id]: 35
   ADI_2001  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         7                       

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

   ADI_2001  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         7                        2.866045                         1.543526
1         8                        4.928303                         2.812831
2         9                        9.620889                         6.062116
3        10                       13.971883                         9.409622
4        11                       17.458742                        12.360560
5        12                       20.383018                        15.060366
6        13                       64.063129                        58.748152
7        14                       66.936392                        61.842979
8        15                      100.000000                       100.000000
Gini Co-Effecient trapezium Rule : 0.07999653658291427
[2011 State Id]: 3  [2011 District Id]: 41
   ADI_2001  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         9                        1.660584            

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

   ADI_2001  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         7                        0.910949                         0.492577
1         8                        2.726762                         1.614707
2        10                        5.262154                         3.573223
3        11                       15.936073                        12.643043
4        12                       45.934331                        40.450377
5        13                       53.847473                        48.396850
6        14                       78.108191                        74.633825
7        15                      100.000000                       100.000000
Gini Co-Effecient trapezium Rule : 0.0705846584212346
[2011 State Id]: 6  [2011 District Id]: 78
   ADI_2001  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         7                       10.953108                         6.916645
1         8                       19.864902             

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

Gini Co-Effecient trapezium Rule : 0.05809034686660707
[2011 State Id]: 29  [2011 District Id]: 581
   ADI_2001  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         7                        2.406316                         1.808827
1         8                       17.856880                        15.082180
2         9                       65.258913                        60.894870
3        10                       88.337406                        85.677865
4        11                       97.458760                        96.452394
5        13                      100.000000                       100.000000
Gini Co-Effecient trapezium Rule : 0.05900200484257257
[2011 State Id]: 29  [2011 District Id]: 582
   ADI_2001  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         7                        1.370385                         0.873770
1         8                       12.553021                         9.022515
2         9                   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

   ADI_2001  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         7                       35.840249                        30.148342
1         8                       50.466805                        44.209673
2         9                       90.767635                        87.796061
3        11                      100.000000                       100.000000
Gini Co-Effecient trapezium Rule : 0.07781317921359406
[2011 State Id]: 33  [2011 District Id]: 611
   ADI_2001  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         8                        0.211688                         0.160496
1         9                       34.281004                        29.219754
2        10                       71.587462                        64.575713
3        11                       76.397488                        69.590110
4        12                       78.971988                        72.517987
5        13                       90.431922          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [9]:
df_2001

,STATE_CODE_2011,DIST_CODE_2011,Gini_2001,Population_sum_2001,Adi_product_population_sum_2001,Average_ADI_per_population_2001,Num_villages_in_District_2001
0,3.0,35.0,0.081987,1190370.0,15207334.0,12.775300,1223.0
1,3.0,37.0,0.049305,909528.0,12677151.0,13.938165,829.0
2,3.0,38.0,0.067285,1084402.0,14958808.0,13.794523,1233.0
3,3.0,39.0,0.092315,459081.0,6063192.0,13.207238,417.0
4,3.0,40.0,0.128257,320338.0,3375632.0,10.537720,355.0
...,...,...,...,...,...,...,...
401,33.0,627.0,0.000000,290.0,2610.0,9.000000,1.0
402,33.0,630.0,0.039200,85383.0,790016.0,9.252615,61.0
403,33.0,631.0,0.045161,448668.0,4225355.0,9.417554,240.0
404,33.0,632.0,0.042210,46228.0,656521.0,14.201804,10.0


In [10]:
df_2001 = df_2001.set_index(['STATE_CODE_2011', 'DIST_CODE_2011'])

In [11]:
df_2001

Gini_2001  ...  Num_villages_in_District_2001
STATE_CODE_2011 DIST_CODE_2011             ...                               
3.0             35.0             0.081987  ...                         1223.0
                37.0             0.049305  ...                          829.0
                38.0             0.067285  ...                         1233.0
                39.0             0.092315  ...                          417.0
                40.0             0.128257  ...                          355.0
...                                   ...  ...                            ...
33.0            627.0            0.000000  ...                            1.0
                630.0            0.039200  ...                           61.0
                631.0            0.045161  ...                          240.0
                632.0            0.042210  ...                           10.0
                633.0            0.045931  ...                           19.0

[406 rows x 5 columns]

In [12]:
df_2001 = df_2001.rename(columns={"Population_sum":"Population_sum_2001", "Adi_product_population_sum": "Adi_product_population_sum_2001", "ADI_per_population": "ADI_per_population_2001","Num_villages_in_District":"Num_villages_in_District_2001"})

In [13]:
df_2001

Gini_2001  ...  Num_villages_in_District_2001
STATE_CODE_2011 DIST_CODE_2011             ...                               
3.0             35.0             0.081987  ...                         1223.0
                37.0             0.049305  ...                          829.0
                38.0             0.067285  ...                         1233.0
                39.0             0.092315  ...                          417.0
                40.0             0.128257  ...                          355.0
...                                   ...  ...                            ...
33.0            627.0            0.000000  ...                            1.0
                630.0            0.039200  ...                           61.0
                631.0            0.045161  ...                          240.0
                632.0            0.042210  ...                           10.0
                633.0            0.045931  ...                           19.0

[406 rows x 5 columns]

In [14]:
df_2011 = getYearDataframe("2011")

Number of States: 14
   ADI_2011  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         7                        0.108869                         0.058256
1         8                        0.248249                         0.143493
2         9                        0.427679                         0.266939
3        10                        3.350797                         2.501461
4        11                        8.359163                         6.712866
5        12                       29.607147                        26.204010
6        13                       54.346065                        50.788568
7        14                       95.389635                        94.713541
8        15                      100.000000                       100.000000
Gini Co-Effecient trapezium Rule : 0.047189402193645216
[2011 State Id]: 3  [2011 District Id]: 35
   ADI_2011  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         7                      

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

   ADI_2011  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         7                        0.017225                         0.008904
1         8                        0.105811                         0.061241
2         9                        0.233260                         0.145949
3        10                        0.907330                         0.643747
4        11                        1.970195                         1.507160
5        12                        9.857592                         8.496930
6        13                       42.549382                        39.882499
7        14                       90.255093                        89.205142
8        15                      100.000000                       100.000000
Gini Co-Effecient trapezium Rule : 0.03346281944561996
[2011 State Id]: 3  [2011 District Id]: 41
   ADI_2011  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         7                        0.006052            

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

   ADI_2011  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         6                        0.016168                         0.007770
1         9                        1.042853                         0.747881
2        10                        2.656189                         2.040116
3        11                       13.065246                        11.211203
4        12                       50.325938                        47.024879
5        13                       86.531282                        84.724136
6        14                       97.845211                        97.411115
7        15                      100.000000                       100.000000
Gini Co-Effecient trapezium Rule : 0.04308126239020985
[2011 State Id]: 6  [2011 District Id]: 79
   ADI_2011  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         8                        0.396767                         0.246273
1         9                        0.798790            

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

   ADI_2011  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         8                       17.710621                        13.018683
1         9                       22.305192                        16.818220
2        10                       36.591847                        29.945473
3        11                       61.556707                        55.178252
4        12                       73.515330                        68.364027
5        13                      100.000000                       100.000000
Gini Co-Effecient trapezium Rule : 0.09026834121803168
[2011 State Id]: 33  [2011 District Id]: 609
   ADI_2011  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         9                       12.915770                        11.667294
1        10                       95.389209                        94.446535
2        12                      100.000000                       100.000000
Gini Co-Effecient trapezium Rule : 0.0201183134660890

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

   ADI_2011  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0        10                       68.335798                        64.613868
1        11                       90.823297                        88.002850
2        12                       92.224805                        89.593058
3        13                       95.506726                        93.627180
4        15                      100.000000                       100.000000
Gini Co-Effecient trapezium Rule : 0.04323550613385013
[2011 State Id]: 33  [2011 District Id]: 621
   ADI_2011  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         8                        0.569897                         0.388914
1         9                        4.004397                         3.025685
2        10                       18.095533                        15.045905
3        11                       61.201628                        55.493976
4        12                       62.838495          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [15]:
df_2011

,STATE_CODE_2011,DIST_CODE_2011,Gini_2011,Population_sum_2011,Adi_product_population_sum_2011,Average_ADI_per_population_2011,Num_villages_in_District_2011
0,3.0,35.0,0.047189,1320850.0,17278863.0,13.081624,1223.0
1,3.0,37.0,0.034979,943005.0,13165768.0,13.961504,829.0
2,3.0,38.0,0.039400,1147816.0,16021021.0,13.957830,1233.0
3,3.0,39.0,0.035722,459948.0,6398959.0,13.912353,417.0
4,3.0,40.0,0.029908,342391.0,4671674.0,13.644266,355.0
...,...,...,...,...,...,...,...
401,33.0,627.0,0.000000,295.0,2360.0,8.000000,1.0
402,33.0,630.0,0.072128,103181.0,994964.0,9.642899,61.0
403,33.0,631.0,0.079975,517708.0,5756588.0,11.119372,240.0
404,33.0,632.0,0.038716,56504.0,784722.0,13.887902,10.0


In [16]:
df_2011 = df_2011.set_index(['STATE_CODE_2011', 'DIST_CODE_2011'])

In [17]:
df_2011

Gini_2011  ...  Num_villages_in_District_2011
STATE_CODE_2011 DIST_CODE_2011             ...                               
3.0             35.0             0.047189  ...                         1223.0
                37.0             0.034979  ...                          829.0
                38.0             0.039400  ...                         1233.0
                39.0             0.035722  ...                          417.0
                40.0             0.029908  ...                          355.0
...                                   ...  ...                            ...
33.0            627.0            0.000000  ...                            1.0
                630.0            0.072128  ...                           61.0
                631.0            0.079975  ...                          240.0
                632.0            0.038716  ...                           10.0
                633.0            0.050380  ...                           19.0

[406 rows x 5 columns]

In [18]:
df_2011 = df_2011.rename(columns={"Population_sum":"Population_sum_2011", "Adi_product_population_sum": "Adi_product_population_sum_2011", "ADI_per_population": "ADI_per_population_2011","Num_villages_in_District":"Num_villages_in_District_2011"})

In [19]:
df_2011

Gini_2011  ...  Num_villages_in_District_2011
STATE_CODE_2011 DIST_CODE_2011             ...                               
3.0             35.0             0.047189  ...                         1223.0
                37.0             0.034979  ...                          829.0
                38.0             0.039400  ...                         1233.0
                39.0             0.035722  ...                          417.0
                40.0             0.029908  ...                          355.0
...                                   ...  ...                            ...
33.0            627.0            0.000000  ...                            1.0
                630.0            0.072128  ...                           61.0
                631.0            0.079975  ...                          240.0
                632.0            0.038716  ...                           10.0
                633.0            0.050380  ...                           19.0

[406 rows x 5 columns]

In [20]:
df_2019 = getYearDataframe("2019")

Number of States: 14
   ADI_2019  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         8                        0.434986                         0.278900
1         9                        2.077725                         1.463836
2        10                        5.413974                         4.137718
3        11                        9.918347                         8.108818
4        12                       54.363236                        50.853969
5        13                       88.175854                        86.083425
6        14                       91.898520                        90.260439
7        15                      100.000000                       100.000000
Gini Co-Effecient trapezium Rule : 0.04744978957826097
[2011 State Id]: 3  [2011 District Id]: 35
   ADI_2019  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         8                        0.353517                         0.203938
1         9                       

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

   ADI_2019  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         8                        0.142331                         0.081602
1         9                        0.301105                         0.184010
2        10                        2.662670                         1.876441
3        11                        6.316348                         4.756716
4        12                       13.989210                        11.355283
5        13                       40.556136                        36.106451
6        14                       40.663527                        36.214198
7        15                      100.000000                       100.000000
Gini Co-Effecient trapezium Rule : 0.04960706978993749
[2011 State Id]: 3  [2011 District Id]: 42
   ADI_2019  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         7                        0.186186                         0.114246
1         8                        3.078960            

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

   ADI_2019  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         9                        1.136032                         0.854246
1        10                        6.640354                         5.453143
2        11                       34.155456                        30.741095
3        12                       78.436581                        75.137698
4        13                       88.704726                        86.290530
5        14                       94.048106                        92.540731
6        15                      100.000000                       100.000000
Gini Co-Effecient trapezium Rule : 0.052087762086158555
[2011 State Id]: 6  [2011 District Id]: 78
   ADI_2019  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         8                        0.327605                         0.208994
1         9                        6.787553                         4.845221
2        10                       19.350465           

...

   ADI_2019  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         6                        0.171860                         0.108974
1         7                       11.520538                         8.504363
2         8                       30.354288                        24.427342
3         9                       57.771227                        50.504425
4        10                       79.941379                        73.934095
5        11                       83.217027                        77.742012
6        12                       95.403499                        93.196576
7        13                       97.006618                        95.399030
8        14                       98.371005                        97.417689
9        15                      100.000000                       100.000000
Gini Co-Effecient trapezium Rule : 0.10017970903693495
[2011 State Id]: 33  [2011 District Id]: 606
   ADI_2019  Cummulative_Population_Percent  Cummulat

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

   ADI_2019  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         7                        7.681939                         5.785690
1         8                       23.075597                        19.035758
2         9                       61.624790                        56.364562
3        10                       89.097008                        85.922899
4        12                      100.000000                       100.000000
Gini Co-Effecient trapezium Rule : 0.07307722737567655
[2011 State Id]: 33  [2011 District Id]: 619
   ADI_2019  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0         8                        5.932065                         4.028809
1         9                       16.197927                        11.872469
2        10                       27.170539                        21.187641
3        11                       44.707788                        37.564665
4        12                       52.854763          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

   ADI_2019  Cummulative_Population_Percent  Cummulative_adi_product_Percent
0        10                        9.722672                         7.893310
1        11                       16.141788                        13.625776
2        12                       74.117388                        70.106469
3        13                       76.855657                        72.996437
4        14                       91.401418                        89.528917
5        15                      100.000000                       100.000000
Gini Co-Effecient trapezium Rule : 0.05451062183943595
[2011 State Id]: 33  [2011 District Id]: 633
Completed state Id : 33
Successfully Completed Everything


In [21]:
df_2019

,STATE_CODE_2011,DIST_CODE_2011,Gini_2019,Population_sum_2019,Adi_product_population_sum_2019,Average_ADI_per_population_2019,Num_villages_in_District_2019
0,3.0,35.0,0.047450,1498899.0,18702023.0,12.477174,1223.0
1,3.0,37.0,0.051354,1010984.0,14019929.0,13.867607,829.0
2,3.0,38.0,0.060786,1238412.0,15122273.0,12.211019,1233.0
3,3.0,39.0,0.048773,466547.0,6538985.0,14.015705,417.0
4,3.0,40.0,0.033202,393191.0,5597018.0,14.234858,355.0
...,...,...,...,...,...,...,...
401,33.0,627.0,0.000000,300.0,3000.0,10.000000,1.0
402,33.0,630.0,0.102306,153887.0,1493935.0,9.708000,61.0
403,33.0,631.0,0.113958,698029.0,7599627.0,10.887265,240.0
404,33.0,632.0,0.000000,72971.0,1094565.0,15.000000,10.0


In [22]:
df_2019 = df_2019.set_index(['STATE_CODE_2011', 'DIST_CODE_2011'])

In [23]:
df_2019

Gini_2019  ...  Num_villages_in_District_2019
STATE_CODE_2011 DIST_CODE_2011             ...                               
3.0             35.0             0.047450  ...                         1223.0
                37.0             0.051354  ...                          829.0
                38.0             0.060786  ...                         1233.0
                39.0             0.048773  ...                          417.0
                40.0             0.033202  ...                          355.0
...                                   ...  ...                            ...
33.0            627.0            0.000000  ...                            1.0
                630.0            0.102306  ...                           61.0
                631.0            0.113958  ...                          240.0
                632.0            0.000000  ...                           10.0
                633.0            0.054511  ...                           19.0

[406 rows x 5 columns]

## Combine All Data

In [24]:
df_combined_gini = df_2001.copy()

In [25]:
df_combined_gini

Gini_2001  ...  Num_villages_in_District_2001
STATE_CODE_2011 DIST_CODE_2011             ...                               
3.0             35.0             0.081987  ...                         1223.0
                37.0             0.049305  ...                          829.0
                38.0             0.067285  ...                         1233.0
                39.0             0.092315  ...                          417.0
                40.0             0.128257  ...                          355.0
...                                   ...  ...                            ...
33.0            627.0            0.000000  ...                            1.0
                630.0            0.039200  ...                           61.0
                631.0            0.045161  ...                          240.0
                632.0            0.042210  ...                           10.0
                633.0            0.045931  ...                           19.0

[406 rows x 5 columns]

In [26]:
df_combined_gini = df_combined_gini.merge(df_2011 , how='inner', left_index=True ,right_index=True ).set_index(df_combined_gini.index)

In [27]:
df_combined_gini

Gini_2001  ...  Num_villages_in_District_2011
STATE_CODE_2011 DIST_CODE_2011             ...                               
3.0             35.0             0.081987  ...                         1223.0
                37.0             0.049305  ...                          829.0
                38.0             0.067285  ...                         1233.0
                39.0             0.092315  ...                          417.0
                40.0             0.128257  ...                          355.0
...                                   ...  ...                            ...
33.0            627.0            0.000000  ...                            1.0
                630.0            0.039200  ...                           61.0
                631.0            0.045161  ...                          240.0
                632.0            0.042210  ...                           10.0
                633.0            0.045931  ...                           19.0

[406 rows x 10 columns]

In [28]:
df_combined_gini = df_combined_gini.merge(df_2019 , how='inner', left_index=True ,right_index=True ).set_index(df_combined_gini.index)

In [29]:
df_combined_gini

Gini_2001  ...  Num_villages_in_District_2019
STATE_CODE_2011 DIST_CODE_2011             ...                               
3.0             35.0             0.081987  ...                         1223.0
                37.0             0.049305  ...                          829.0
                38.0             0.067285  ...                         1233.0
                39.0             0.092315  ...                          417.0
                40.0             0.128257  ...                          355.0
...                                   ...  ...                            ...
33.0            627.0            0.000000  ...                            1.0
                630.0            0.039200  ...                           61.0
                631.0            0.045161  ...                          240.0
                632.0            0.042210  ...                           10.0
                633.0            0.045931  ...                           19.0

[406 rows x 15 columns]

In [30]:
pickle.dump(df_combined_gini,open('/content/drive/MyDrive/Scripts_and_Data_Transfer/Latest_script_Data_for_hyothesis_testing_March_02_2022/combined_gini_2001_2011_2019_from_without_temporal_arch2_predicted_data.pickle','wb'))

In [31]:
df_combined_gini

Gini_2001  ...  Num_villages_in_District_2019
STATE_CODE_2011 DIST_CODE_2011             ...                               
3.0             35.0             0.081987  ...                         1223.0
                37.0             0.049305  ...                          829.0
                38.0             0.067285  ...                         1233.0
                39.0             0.092315  ...                          417.0
                40.0             0.128257  ...                          355.0
...                                   ...  ...                            ...
33.0            627.0            0.000000  ...                            1.0
                630.0            0.039200  ...                           61.0
                631.0            0.045161  ...                          240.0
                632.0            0.042210  ...                           10.0
                633.0            0.045931  ...                           19.0

[406 rows x 15 columns]